In [ ]:
%pip install datacontract-cli

In [ ]:
def get_absolute_path(*relative_parts):
    import os
    if 'dbutils' in globals():
        # dbutils is available to the entry point when run in Databricks
        base_dir = os.path.dirname(dbutils.notebook.entry_point.getDbutils().notebook().getContext().notebookPath().get()) # type: ignore
        path = os.path.normpath(os.path.join(base_dir, *relative_parts))
        return path if path.startswith("/Workspace") else "/Workspace" + path
    else:
        return os.path.join(*relative_parts)

In [ ]:
import sys
from datacontract.data_contract import DataContract
from helpers.spark import get_spark

run_verification = dbutils.jobs.taskValues.get(taskKey = "main_task", key = "verify_promise")
if run_verification is None | run_verification == False:
    print("Skipping verification due to run_verification value")
else:
    data_contract_file = get_absolute_path("..", "datacontract.yaml")
    data_contract = DataContract(
        data_contract_file=data_contract_file, 
        spark=get_spark(),
    )

    run = data_contract.test()

    print("Data Contract test result:", run.result)
    print("Run details:",  run)

    if not run.has_passed():
        sys.exit(1)